# 程序说明
时间：2016年11月16日

说明：说明：这是一个使用卷积网络在CIFAR10数据集上做分类的程序，其中使用了数据增强。

数据集：CIFAR10

## 1.加载keras模块

In [1]:
from __future__ import print_function
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.optimizers import SGD
from keras.utils import np_utils

Using TensorFlow backend.


## 2.变量初始化

In [2]:
batch_size = 32
nb_classes = 10
nb_epoch = 20
data_augmentation = True

# input image dimensions
img_rows, img_cols = 32, 32
# the CIFAR10 images are RGB
img_channels = 3

## 3.准备数据

In [3]:
# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = cifar10.load_data()
print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

X_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


### 转换类标号

In [4]:
# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)

## 4.建立模型
### 使用Sequential（）

In [5]:
model = Sequential()

model.add(Convolution2D(32, 3, 3, border_mode='same',
                        input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(Convolution2D(32, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Convolution2D(64, 3, 3, border_mode='same'))
model.add(Activation('relu'))
model.add(Convolution2D(64, 3, 3))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(nb_classes))
model.add(Activation('softmax'))

### 打印模型

In [6]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_1 (Convolution2D)  (None, 32, 32, 32)    896         convolution2d_input_1[0][0]      
____________________________________________________________________________________________________
activation_1 (Activation)        (None, 32, 32, 32)    0           convolution2d_1[0][0]            
____________________________________________________________________________________________________
convolution2d_2 (Convolution2D)  (None, 30, 30, 32)    9248        activation_1[0][0]               
____________________________________________________________________________________________________
activation_2 (Activation)        (None, 30, 30, 32)    0           convolution2d_2[0][0]            
___________________________________________________________________________________________

## 5.训练与评估
### 编译模型

In [7]:
# let's train the model using SGD + momentum (how original).
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

### 数据压缩为0~1之间

In [8]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


### 数据增强
数据增强使用的ImageDataGenerator这个函数。

In [9]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, Y_train,
              batch_size=batch_size,
              nb_epoch=nb_epoch,
              validation_data=(X_test, Y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')

    # 这将做预处理和实时数据增加
    datagen = ImageDataGenerator(
        featurewise_center=False,  # 在数据集上将输入平均值设置为0
        samplewise_center=False,  # 将每个样本均值设置为0
        featurewise_std_normalization=False,  # 将输入除以数据集的std
        samplewise_std_normalization=False,  # 将每个输入除以其std
        zca_whitening=False,  # 应用ZCA白化
        rotation_range=0,  # 在一个范围下随机旋转图像(degrees, 0 to 180)
        width_shift_range=0.1,  # 水平随机移位图像（总宽度的分数）
        height_shift_range=0.1,  # 随机地垂直移动图像（总高度的分数）
        horizontal_flip=True,  # 随机翻转图像
        vertical_flip=False)  # 随机翻转图像

    # 计算特征方向归一化所需的数量
    # (std, mean, and principal components if ZCA whitening is applied)
    datagen.fit(X_train)

    # fit the model on the batches generated by datagen.flow()
    model.fit_generator(datagen.flow(X_train, Y_train,
                        batch_size=batch_size),
                        samples_per_epoch=X_train.shape[0],
                        nb_epoch=nb_epoch,
                        validation_data=(X_test, Y_test))

Using real-time data augmentation.
Epoch 1/20
50000/50000 [==============================] - 37s - loss: 2.3037 - acc: 0.1010 - val_loss: 2.3034 - val_acc: 0.1000
Epoch 2/20
50000/50000 [==============================] - 37s - loss: 2.3035 - acc: 0.0991 - val_loss: 2.3033 - val_acc: 0.1000
Epoch 3/20
50000/50000 [==============================] - 38s - loss: 2.3034 - acc: 0.0977 - val_loss: 2.3030 - val_acc: 0.1000
Epoch 4/20
50000/50000 [==============================] - 36s - loss: 2.3035 - acc: 0.0975 - val_loss: 2.3036 - val_acc: 0.1000
Epoch 5/20
50000/50000 [==============================] - 37s - loss: 2.3035 - acc: 0.0986 - val_loss: 2.3028 - val_acc: 0.1000
Epoch 6/20
50000/50000 [==============================] - 36s - loss: 2.3034 - acc: 0.0991 - val_loss: 2.3031 - val_acc: 0.1000
Epoch 7/20
50000/50000 [==============================] - 37s - loss: 2.3033 - acc: 0.1004 - val_loss: 2.3030 - val_acc: 0.1000
Epoch 8/20
50000/50000 [==============================] - 38s - loss: